# Playing Around with pvlib

In [56]:
#IMPORTS
import pandas as pd
import os
import numpy as np
import pysolar
import pvlib
import datetime
import pytz
import math
from dateutil import tz

print(os.listdir())

['.statistical_modeling 8.ipynb.icloud', '.irradience_fitter 2.ipynb.icloud', '.Data_Viz.ipynb.icloud', '.statistical_modeling 10.ipynb.icloud', 'data_munger.py', 'Testing_Data', 'Untitled1.ipynb', '.irradience_fitter 3.ipynb.icloud', '.DS_Store', '.statistical_modeling 9.ipynb.icloud', '.statistical_modeling 11.ipynb.icloud', 'Untitled.ipynb', '.irradience_fitter.ipynb.icloud', '.Data_Viz 2.ipynb.icloud', '.statistical_modeling 3.ipynb.icloud', 'playing_with_pvlib.ipynb', '.statistical_modeling 4.ipynb.icloud', 'statistical_modeling.py', 'Raw_Data', '.statistical_modeling 5.ipynb.icloud', '.statistical_modeling 2.ipynb.icloud', 'README.md', '.statistical_modeling 7.ipynb.icloud', 'psm_VA_Charlottesville2018cloud_output.csv', '.statistical_modeling 6.ipynb.icloud', '.statistical_modeling.ipynb.icloud', '.ipynb_checkpoints', '.git', '.statistical_modeling 12.ipynb.icloud']


In [2]:
_Curr_Dir = os.getcwd()
_Testing_Data_Dir = os.path.join(_Curr_Dir, 'Testing_Data')
print(os.listdir(_Testing_Data_Dir))

['cloud_output_darksky_2009.csv', 'cloud_output_darksky_2020.csv', 'cloud_output_darksky_2008.csv', 'data_munger.py', 'cloud_output_darksky_2018.csv', 'cloud_output_darksky_2019.csv', 'psm_testing_data2018_copy.csv', 'Charlottesville_psm_2018_cloud_output_darksky_2018.csv', 'psm_testing_data2009.csv', 'psm_testing_data2008.csv', 'psm_testing_data2018.csv', 'Austin_psm_2018.csv', 'psm_testing_data2017.csv', 'psm_testing_data2016.csv', 'psm_testing_data2014.csv', 'Austin_psm_2018_cloud_output_darksky_2018.csv', 'psm_testing_data2015.csv', 'psm_testing_data2011.csv', 'psm_testing_data2010.csv', 'psm_testing_data2012.csv', 'psm_testing_data2007.csv', 'psm_testing_data2013.csv', 'cloud_output_darksky_2014.csv', 'cloud_output_darksky_2015.csv', 'cloud_output_darksky_2017.csv', 'cloud_output_darksky_2016.csv', 'cloud_output_darksky_2012.csv', 'Charlottesville_psm_2018.csv', 'cloud_output_darksky_2013.csv', 'cloud_output_darksky_2007.csv', 'cloud_output_darksky_2011.csv', 'cloud_output_darksky

In [31]:
def get_dni(ghi, lat, lon, day):
    # pvlib.irradiance.disc(ghi, solar_zenith, datetime_or_doy, pressure=101325, min_cos_zenith=0.065, max_zenith=87, max_airmass=12) 
    utcTime, date = _make_good_date(day[0], day[1], day[2], day[3], day[4])
    solar_altitude = pysolar.solar.get_altitude(lat,lon,date)
    solar_zenith = 90 - solar_altitude
    DNI = pvlib.irradiance.disc(ghi, solar_zenith, date)
    return DNI

In [32]:
def _make_good_date(year,month,day,hour,minute):
    date = datetime.datetime(year, month, day, hour, minute)
    date = pytz.timezone('US/Eastern').localize(date)
    utcTime = date.astimezone(pytz.timezone('UTC'))
    return utcTime, date

In [35]:
def make_df_nonzero(df):
    df_no_zero = df.loc[df['DHI']!= 0 ]
    assert len(df_no_zero['DHI'])==len(df_no_zero['GHI']), "Some DHI 0s where GHI is not 0"
    print(df_no_zero)
    return df_no_zero 

In [34]:
# date = datetime.datetime(2018, 1, 22, 15, 30)
# date = pytz.timezone('US/Eastern').localize(date)
# print(date)
# utcTime = date.astimezone(pytz.timezone('UTC'))
# print(utcTime)

In [53]:
#Get 2018 Data
df1_2018 = pd.read_csv(os.path.join(_Testing_Data_Dir, 'psm_testing_data2018.csv'))
df1 = df1_2018[['Month','Day','Hour','Minute','GHI', 'DNI', 'DHI']]
df1 = make_df_nonzero(df1)
print(df1)
print(df1['Year'])

      Month  Day  Hour  Minute  GHI  DNI  DHI
14        1    1    14      30   14   50   12
15        1    1    15      30  106  301   53
16        1    1    16      30  274  742   60
17        1    1    17      30  376  860   64
18        1    1    18      30  440  976   56
...     ...  ...   ...     ...  ...  ...  ...
8754     12   31    18      30  382  812   64
8755     12   31    19      30  316  569  101
8756     12   31    20      30  257  525   89
8757     12   31    21      30  154  385   69
8758     12   31    22      30   50  191   33

[4494 rows x 7 columns]
      Month  Day  Hour  Minute  GHI  DNI  DHI
14        1    1    14      30   14   50   12
15        1    1    15      30  106  301   53
16        1    1    16      30  274  742   60
17        1    1    17      30  376  860   64
18        1    1    18      30  440  976   56
...     ...  ...   ...     ...  ...  ...  ...
8754     12   31    18      30  382  812   64
8755     12   31    19      30  316  569  101
8756     

KeyError: 'Year'

In [61]:
preds = []
resids = []
actuals = []
for i in range(len(df1)):
    lat = 38.0086
    lon = -78.4532
    year = 2018
    month = (df1.iloc[i]['Month'])
    day = df1.iloc[i]['Day']
    hour = df1.iloc[i]['Hour']
    minute = df1.iloc[i]['Minute']
    date = [year, month, day, hour, minute]
    ghi = df1.iloc[i]['GHI']
    dni_actual = df1.iloc[i]['DNI']
    dni_pred = get_dni(ghi, lat, lon, date)['dni']
    preds.append(dni_pred)
    resids.append(abs(dni_actual-dni_pred))
    actuals.append(dni_actual)
    
    
    
    
    

In [62]:
print(resids)

[50.0, 254.85145988268346, 392.77183267550146, 860.0, 976.0, 967.0, 926.0, 830.0, 597.0, 123.0, 373.61202378883274, 360.64176739433697, 347.0, 54.0, 78.0, 196.0, 239.0, 46.0, 18.0, 116.0, 157.44579297989486, 528.0, 531.0, 816.0, 295.0, 212.0, 246.0, 4.0, 186.256564259929, 262.1916458660297, 829.0, 694.0, 604.0, 803.0, 437.0, 401.0, 24.0, 193.13990807872912, 172.11384178635285, 469.0, 657.0, 653.0, 630.0, 171.0, 96.0, 110.0, 369.6281857148248, 370.53550454596626, 643.0, 591.0, 585.0, 91.0, 480.0, 260.0, 140.0, 310.09666085257174, 242.14544827298107, 811.0, 406.0, 727.0, 615.0, 544.0, 185.0, 109.0, 386.1013425236568, 130.72153387409372, 546.0, 642.0, 426.0, 597.0, 553.0, 368.0, 124.0, 362.46235431164433, 203.2692089038198, 649.0, 701.0, 594.0, 374.0, 385.0, 390.0, 96.0, 282.78215108335434, 146.2061070389305, 160.0, 187.0, 140.0, 167.0, 0.0, 0.0, 25.0, 0.0, 167.30101749712435, 855.0, 669.0, 933.0, 897.0, 804.0, 618.0, 47.0, 175.90460521803513, 179.4467969256924, 520.0, 355.0, 498.0, 435.0

In [65]:
print(np.mean(resids))

321.97722888917065


In [66]:
for i in range(20):
    print(preds[i], actuals[i], resids[i])

0.0 50 50.0
46.14854011731654 301 254.85145988268346
349.22816732449854 742 392.77183267550146
0.0 860 860.0
0.0 976 976.0
0.0 967 967.0
0.0 926 926.0
0.0 830 830.0
0.0 597 597.0
0.0 123 123.0
259.38797621116726 633 373.61202378883274
355.35823260566303 716 360.64176739433697
0.0 347 347.0
0.0 54 54.0
0.0 78 78.0
0.0 196 196.0
0.0 239 239.0
0.0 46 46.0
0.0 18 18.0
0.0 116 116.0
